In [155]:
"""
URL: https://www.larepublica.co/

# Accediendo alos links de las notificas titulares
$x('//h2[@class="headline"]/a/@href').map(x => x.value)

# Accediendo a los links de las notificas pequeñas
$x('//div[@class="news V_Title_Img"]/h2/a/@href').map(x => x.value)

////////////////////////////////////////////////////////////////////

# Accediendo al titulo detalle noticia
https://www.larepublica.co/economia/gobierno-nacional-levanta-restriccion-a-vehiculos-de-carga-para-este-domingo-25-de-abril-3159071
$x('//div[@class="row OpeningPostNormal"]/div[2]/div/h2/span/text()').map(x => x.wholeText)

# Accedinedo al resumen de la notifica
$x('//div[@class="lead"]/p/text()').map(x => x.wholeText)

# Accediendo a todo el texto de los parrafos del contenido
# p[not(@class)] // accede a todos los p que no tengan ninguna clase
$x('//div[@class="html-content"]/p[not(@class)]/text()').map(x => x.wholeText)

# Accediendo a la url de la imagen principal
$x('//div[@class="img-principal open-gallery"]/img/@data-original')

"""



'\nURL: https://www.larepublica.co/\n\n# Accediendo alos links de las notificas titulares\n$x(\'//h2[@class="headline"]/a/@href\').map(x => x.value)\n\n# Accediendo a los links de las notificas pequeñas\n$x(\'//div[@class="news V_Title_Img"]/h2/a/@href\').map(x => x.value)\n\n////////////////////////////////////////////////////////////////////\n\n# Accediendo al titulo detalle noticia\nhttps://www.larepublica.co/economia/gobierno-nacional-levanta-restriccion-a-vehiculos-de-carga-para-este-domingo-25-de-abril-3159071\n$x(\'//div[@class="row OpeningPostNormal"]/div[2]/div/h2/span/text()\').map(x => x.wholeText)\n\n# Accedinedo al resumen de la notifica\n$x(\'//div[@class="lead"]/p/text()\').map(x => x.wholeText)\n\n# Accediendo a todo el texto de los parrafos del contenido\n# p[not(@class)] // accede a todos los p que no tengan ninguna clase\n$x(\'//div[@class="html-content"]/p[not(@class)]/text()\').map(x => x.wholeText)\n\n# Accediendo a la url de la imagen principal\n$x(\'//div[@class

In [156]:
# pip install -r requirements.txt

# Imports
import requests
# .html permite ejecutar sobre el html optenido sentncias de xpath
import lxml.html as html
import datetime
import pandas as pd

In [157]:
# Variables globales
HOME_URL = 'https://www.larepublica.co/'
XPATH_LINK_TO_ARTICLE = '//div[@class="news V_Title_Img"]/text-fill/a/@href'
XPATH_TITLE = '//div[@class="row OpeningPostNormal"]/div[2]/div/text-fill/span/text()'
XPATH_SUMMARY = '//div[@class="lead"]/p/text()'
XPATH_BODY = '//div[@class="html-content"]/p[not(@class)]/text()'
XPATH_IMAGE = '//div[@class="img-principal open-gallery"]/img/@data-original'
OUTPUT_DIR = './outputs/la_republica/'


In [158]:
def save_file(data, today):
    df = pd.DataFrame(data, columns=['title', 'summary', 'body', 'image'])
    df.to_csv(OUTPUT_DIR + today + '.csv', sep='|')


In [159]:
#
def parse_home():
    try:
        # Obtenemos todo el html del request
        response = requests.get(HOME_URL)

        if response.status_code == 200:
            # Almacenando todo el html, y decodificando a utf8
            home_content = response.content.decode('utf-8')

            file_home = open(OUTPUT_DIR + 'home_republica.html', 'a')
            file_home.write(home_content)

            # Parseando el documento html para operar con XPATH
            parsed = html.fromstring(home_content)

            # Ejecutando la sentencia de xpath y obteniendo los links
            links_to_notices = parsed.xpath(XPATH_LINK_TO_ARTICLE)
            print(links_to_notices)

            # Fecha formateada
            today = datetime.date.today().strftime('%d-%m-%Y')
            print("today >>", today)

            notices = []
            for link in links_to_notices:
                notice = parse_notice(link)
                print(notice)
                notices.append(notice)
            save_file(notices, today)

        else:
            raise ValueError(f'Error : {response.status_code}')

    except ValueError as ve:
        print(ve)


In [160]:
def parse_notice(link):
    try:
        # Obtenemos todo el html del request
        response = requests.get(link)
        # Almacenando todo el html, y decodificando a utf8
        home_content = response.content.decode('utf-8')
        # Parseando el documento html para operar con XPATH
        parsed = html.fromstring(home_content)
        # Ejecutando la sentencia de xpath y obteniendo los links
        # Se accede a [0] por que solo requerimos la primer posicion
        title = parsed.xpath(XPATH_TITLE)[0]
        # Reemplazando si hay "" por vacio
        title = title.replace('\"', '')

        summary = parsed.xpath(XPATH_SUMMARY)[0] if len(parsed.xpath(XPATH_SUMMARY)) > 0 else ""
        body = parsed.xpath(XPATH_BODY)[0]
        image = parsed.xpath(XPATH_IMAGE)[0]

        return {
            'title': title,
            'summary': summary,
            'body': body,
            'image': image,
        }

    except ValueError as ve:
        print(ve)






In [161]:
def run():
    parse_home()


run()




['https://www.larepublica.co/economia/el-pais-registra-la-mayor-cifra-de-ocupacion-de-camas-uci-tras-superar-80-de-la-capacidad-3159182', 'https://www.larepublica.co/economia/gobierno-anuncia-que-entre-manana-y-el-martes-arriban-mas-de-dos-millones-de-vacunas-3159079', 'https://www.larepublica.co/economia/minsalud-reporto-que-en-las-ultimas-24-horas-se-detectaron-16730-casos-de-covid-19-3159144', 'https://www.larepublica.co/economia/bogota-se-prepara-para-tercera-ola-de-contagios-con-mas-hospitales-de-campana-y-tele-uci-3159106', 'https://www.larepublica.co/globoeconomia/colombia-ocupo-el-puesto-41-del-indice-mundial-de-miseria-2020-que-realiza-hanke-3159122', 'https://www.larepublica.co/economia/gobierno-nacional-levanta-restriccion-a-vehiculos-de-carga-para-este-domingo-25-de-abril-3159071']
today >> 24-04-2021
{'title': 'El país registra la mayor cifra de ocupación de camas UCI tras superar 80% de la capacidad', 'summary': 'En departamentos críticos como Antioquia que tiene una ocup